In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('data_1000.csv')

In [3]:
data=df[['correct_answ','bleu_score','levenstein_sim','cosine_sim','jaccard_sim']]

In [4]:
data.head(10)

,correct_answ,bleu_score,levenstein_sim,cosine_sim,jaccard_sim
0,0,0.000000,0.044444,0.170715,0.150000
1,1,0.263158,0.500000,0.673710,0.642857
2,1,1.000000,1.000000,1.000000,1.000000
3,1,1.000000,1.000000,1.000000,0.818182
4,1,0.263158,0.500000,0.673710,0.642857
5,1,0.333333,0.821429,1.000000,0.722222
6,1,1.000000,0.888889,0.771931,0.833333
7,1,0.263158,0.500000,0.673710,0.642857
8,1,0.322581,0.920000,0.435366,0.736842
9,1,1.000000,1.000000,1.000000,1.000000


In [5]:
data['cos_lev']=data['cosine_sim']+data['levenstein_sim']
data['cos_bleu']=data['cosine_sim']+data['bleu_score']
data['cos_jac']=data['cosine_sim']+data['jaccard_sim']
data['lev_bleu']=data['levenstein_sim']+data['bleu_score']
data['lev_jac']=data['levenstein_sim']+data['jaccard_sim']
data['bleu_jac']=data['bleu_score']+data['jaccard_sim']

c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [6]:
X=data[['bleu_score','levenstein_sim','cosine_sim','jaccard_sim','cos_lev','cos_bleu','cos_jac','lev_bleu','lev_jac','bleu_jac']]
y=data[['correct_answ']]

In [7]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(data, data["correct_answ"]):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]

In [11]:
print(data["correct_answ"].value_counts() / len(data))
print(strat_train_set["correct_answ"].value_counts() / len(strat_train_set))
print(strat_test_set["correct_answ"].value_counts() / len(strat_test_set))

1    0.791749
0    0.208251
Name: correct_answ, dtype: float64
1    0.791155
0    0.208845
Name: correct_answ, dtype: float64
1    0.794118
0    0.205882
Name: correct_answ, dtype: float64


In [13]:
training = strat_train_set.copy()
testing=strat_test_set.copy()

In [14]:
X_training=training[['bleu_score','levenstein_sim','cosine_sim','jaccard_sim','cos_lev','cos_bleu','cos_jac','lev_bleu','lev_jac','bleu_jac']]
y_training=training[['correct_answ']]
X_testing=testing[['bleu_score','levenstein_sim','cosine_sim','jaccard_sim','cos_lev','cos_bleu','cos_jac','lev_bleu','lev_jac','bleu_jac']]
y_testing=testing[['correct_answ']]

In [15]:
from sklearn.gaussian_process import GaussianProcessClassifier
clf = GaussianProcessClassifier().fit(X_training,y_training)
y_training_prediction = clf.predict(X_training)
y_testing_prediction = clf.predict(X_testing)

c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score
from sklearn.base import clone
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
skfolds = StratifiedKFold(n_splits=3, random_state=42)
for train_index, test_index in skfolds.split(X_training, y_training):
    clone_clf = clone(clf)
    X_train_folds = X_training.iloc[train_index]
    y_train_folds = y_training.iloc[train_index]
    X_test_fold = X_training.iloc[test_index]
    y_test_fold = y_training.iloc[test_index]
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    recall_score(y_test_fold,y_pred)
    print(confusion_matrix(y_test_fold,y_pred))
    print('precision_score: ',precision_score(y_test_fold,y_pred))
    print('recall_score: ',recall_score(y_test_fold,y_pred))
    print('f1_socre: ',f1_score(y_test_fold, y_pred))
    print('')
    print('\n')

c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[ 28  29]
 [  3 212]]
precision_score:  0.8796680497925311
recall_score:  0.986046511627907
f1_socre:  0.9298245614035088





c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[ 42  15]
 [ 14 201]]
precision_score:  0.9305555555555556
recall_score:  0.9348837209302325
f1_socre:  0.9327146171693735





c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[ 43  13]
 [ 10 204]]
precision_score:  0.9400921658986175
recall_score:  0.9532710280373832
f1_socre:  0.9466357308584686





In [17]:
# testing data validation
from sklearn.metrics import confusion_matrix
print('confusion_matrix: \n',confusion_matrix(y_testing,y_testing_prediction))
print('precision_score: ',precision_score(y_testing,y_testing_prediction))
print('recall_score: ',recall_score(y_testing,y_testing_prediction))
print('f1_socre: ',f1_score(y_testing,y_testing_prediction))
from sklearn.model_selection import cross_val_score
print('accuracy: ',cross_val_score(clf, X_testing, y_testing, cv=3, scoring="accuracy"))

confusion_matrix: 
 [[ 28  14]
 [ 12 150]]
precision_score:  0.9146341463414634
recall_score:  0.9259259259259259
f1_socre:  0.9202453987730062
accuracy:  [0.88235294 0.76470588 0.89705882]


c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [33]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier().fit(X_training,y_training)
y_training_prediction = clf.predict(X_training)
y_testing_prediction = clf.predict(X_testing)

c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [34]:
skfolds = StratifiedKFold(n_splits=3, random_state=42)
for train_index, test_index in skfolds.split(X_training, y_training):
    clone_clf = clone(clf)
    X_train_folds = X_training.iloc[train_index]
    y_train_folds = y_training.iloc[train_index]
    X_test_fold = X_training.iloc[test_index]
    y_test_fold = y_training.iloc[test_index]
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    recall_score(y_test_fold,y_pred)
    print(confusion_matrix(y_test_fold,y_pred))
    print('precision_score: ',precision_score(y_test_fold,y_pred))
    print('recall_score: ',recall_score(y_test_fold,y_pred))
    print('f1_socre: ',f1_score(y_test_fold, y_pred))
    print('')
    print('\n')

c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[ 34  23]
 [  8 207]]
precision_score:  0.9
recall_score:  0.9627906976744186
f1_socre:  0.9303370786516855





c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[ 41  16]
 [ 18 197]]
precision_score:  0.9248826291079812
recall_score:  0.9162790697674419
f1_socre:  0.9205607476635514



[[ 34  22]
 [ 12 202]]
precision_score:  0.9017857142857143
recall_score:  0.9439252336448598
f1_socre:  0.9223744292237444





c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [35]:
print('confusion_matrix: \n',confusion_matrix(y_testing,y_testing_prediction))
print('precision_score: ',precision_score(y_testing,y_testing_prediction))
print('recall_score: ',recall_score(y_testing,y_testing_prediction))
print('f1_socre: ',f1_score(y_testing,y_testing_prediction))
from sklearn.model_selection import cross_val_score
print('accuracy: ',cross_val_score(clf, X_testing, y_testing, cv=3, scoring="accuracy"))

confusion_matrix: 
 [[ 28  14]
 [ 11 151]]
precision_score:  0.9151515151515152
recall_score:  0.9320987654320988
f1_socre:  0.9235474006116208


c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
c:\users\schaa\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_net

accuracy:  [0.89705882 0.79411765 0.86764706]
